## Extracting and Transforming the Data

### Why carry out this process?
- To deploy this code into AWS Lambda Service
- For Automating the entire data pipeline 

### Installing necessary libraries

In [1]:
!pip install spotipy

  Obtaining dependency information for redis>=3.5.3 from https://files.pythonhosted.org/packages/0b/34/a01250ac1fc9bf9161e07956d2d580413106ce02d5591470130a25c599e3/redis-5.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for async-timeout>=4.0.2 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/250.3 kB ? eta -:--:--
   ---------------------------------------- 250.3/250.3 kB 7.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importing libraries

In [35]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

### Authentication & Authorisation

In [9]:
# Linking the Client Id & Client Secret
# For Authentication purposes
client_credentials_manager = SpotifyClientCredentials(client_id="b2c2eaeb6d1c470284266d6ac725d147", client_secret="3e4d41b59dde4ff29b02e5d8ffbaaef3")

In [11]:
# For Authorisation and object to extract data

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Data Exploration

In [12]:
# Connection with the Spotify playist (Top Songs - Global)
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF"

In [17]:
playlist_URI = playlist_link.split("/")[-1]

In [18]:
data = sp.playlist_tracks(playlist_URI)

In [22]:
# To identify how many albums are there
len(data['items'])

50

In [23]:
data['items'][0]['track']['album']['id']

'2IoSjweCu07y9mv5H1Cj3v'

In [24]:
data['items'][0]['track']['album']['name']

'La Diabla'

In [25]:
data['items'][0]['track']['album']['release_date']

'2023-11-30'

In [26]:
data['items'][0]['track']['album']['total_tracks']

1

In [27]:
data['items'][0]['track']['album']['external_urls']['spotify']

'https://open.spotify.com/album/2IoSjweCu07y9mv5H1Cj3v'

### Consolidating Data from Various Albums, Artists and Songs

In [31]:
# Building a set of information across all albums 
# Extracting specific data among all the albums

album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    
    #Converting the extracted values into a dictionary for dataframe
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                        'total_tracks':album_total_tracks,'url':album_url}
    album_list.append(album_element)

album_list

[{'album_id': '2IoSjweCu07y9mv5H1Cj3v',
  'name': 'La Diabla',
  'release_date': '2023-11-30',
  'total_tracks': 1,
  'url': 'https://open.spotify.com/album/2IoSjweCu07y9mv5H1Cj3v'},
 {'album_id': '2B9amdrHDIKOoEiPUfZGtb',
  'name': 'yes, and?',
  'release_date': '2024-01-12',
  'total_tracks': 1,
  'url': 'https://open.spotify.com/album/2B9amdrHDIKOoEiPUfZGtb'},
 {'album_id': '3UOV8XvCwMKaATRNXrYCjN',
  'name': 'greedy',
  'release_date': '2023-09-15',
  'total_tracks': 1,
  'url': 'https://open.spotify.com/album/3UOV8XvCwMKaATRNXrYCjN'},
 {'album_id': '6VCO0fDBGbRW8mCEvV95af',
  'name': 'Lovin On Me',
  'release_date': '2023-11-10',
  'total_tracks': 1,
  'url': 'https://open.spotify.com/album/6VCO0fDBGbRW8mCEvV95af'},
 {'album_id': '7tzVd1fwkxsorytCBjEJkU',
  'name': 'The Idol Episode 4 (Music from the HBO Original Series)',
  'release_date': '2023-06-23',
  'total_tracks': 3,
  'url': 'https://open.spotify.com/album/7tzVd1fwkxsorytCBjEJkU'},
 {'album_id': '2Cn1d2KgbkAqbZCJ1RzdkA',


In [33]:
# For details regarding artists
artist_list = []
for row in data['items']:
    for key, value in row.items():
        if key == "track":
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'], 'artist_name':artist['name'], 'external_url': artist['href']}
                artist_list.append(artist_dict)

artist_list

[{'artist_id': '3Me35AWHCGqW4sZ7bWWJt1',
  'artist_name': 'Xavi',
  'external_url': 'https://api.spotify.com/v1/artists/3Me35AWHCGqW4sZ7bWWJt1'},
 {'artist_id': '66CXWjxzNUsdJxJ2JdwvnR',
  'artist_name': 'Ariana Grande',
  'external_url': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR'},
 {'artist_id': '45dkTj5sMRSjrmBSBeiHym',
  'artist_name': 'Tate McRae',
  'external_url': 'https://api.spotify.com/v1/artists/45dkTj5sMRSjrmBSBeiHym'},
 {'artist_id': '2LIk90788K0zvyj2JJVwkJ',
  'artist_name': 'Jack Harlow',
  'external_url': 'https://api.spotify.com/v1/artists/2LIk90788K0zvyj2JJVwkJ'},
 {'artist_id': '1Xyo4u8uXC1ZmMpatF05PJ',
  'artist_name': 'The Weeknd',
  'external_url': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ'},
 {'artist_id': '250b0Wlc5Vk0CoUsaCY84M',
  'artist_name': 'JENNIE',
  'external_url': 'https://api.spotify.com/v1/artists/250b0Wlc5Vk0CoUsaCY84M'},
 {'artist_id': '1pBLC0qVRTB5zVMuteQ9jJ',
  'artist_name': 'Lily-Rose Depp',
  'external_url': 

In [34]:
# Extracting details regarding Song list
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_element = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                    'popularity':song_popularity,'song_added':song_added,'album_id':album_id,
                    'artist_id':artist_id
                   }
    song_list.append(song_element)

song_list

[{'song_id': '0R6NfOiLzLj4O5VbYSJAjf',
  'song_name': 'La Diabla',
  'duration_ms': 172264,
  'url': 'https://open.spotify.com/track/0R6NfOiLzLj4O5VbYSJAjf',
  'popularity': 99,
  'song_added': '2024-01-26T16:09:07Z',
  'album_id': '2IoSjweCu07y9mv5H1Cj3v',
  'artist_id': '3Me35AWHCGqW4sZ7bWWJt1'},
 {'song_id': '7gaA3wERFkFkgivjwbSvkG',
  'song_name': 'yes, and?',
  'duration_ms': 214994,
  'url': 'https://open.spotify.com/track/7gaA3wERFkFkgivjwbSvkG',
  'popularity': 96,
  'song_added': '2024-01-26T16:09:07Z',
  'album_id': '2B9amdrHDIKOoEiPUfZGtb',
  'artist_id': '66CXWjxzNUsdJxJ2JdwvnR'},
 {'song_id': '3rUGC1vUpkDG9CZFHMur1t',
  'song_name': 'greedy',
  'duration_ms': 131872,
  'url': 'https://open.spotify.com/track/3rUGC1vUpkDG9CZFHMur1t',
  'popularity': 100,
  'song_added': '2024-01-26T16:09:07Z',
  'album_id': '3UOV8XvCwMKaATRNXrYCjN',
  'artist_id': '45dkTj5sMRSjrmBSBeiHym'},
 {'song_id': '4xhsWYTOGcal8zt0J161CU',
  'song_name': 'Lovin On Me',
  'duration_ms': 138411,
  'url':

### Dataframe Initialization - Albums, Artists & Songs

In [37]:
album_df = pd.DataFrame.from_dict(album_list)
album_df.head()

,album_id,name,release_date,total_tracks,url
0,2IoSjweCu07y9mv5H1Cj3v,La Diabla,2023-11-30,1,https://open.spotify.com/album/2IoSjweCu07y9mv...
1,2B9amdrHDIKOoEiPUfZGtb,"yes, and?",2024-01-12,1,https://open.spotify.com/album/2B9amdrHDIKOoEi...
2,3UOV8XvCwMKaATRNXrYCjN,greedy,2023-09-15,1,https://open.spotify.com/album/3UOV8XvCwMKaATR...
3,6VCO0fDBGbRW8mCEvV95af,Lovin On Me,2023-11-10,1,https://open.spotify.com/album/6VCO0fDBGbRW8mC...
4,7tzVd1fwkxsorytCBjEJkU,The Idol Episode 4 (Music from the HBO Origina...,2023-06-23,3,https://open.spotify.com/album/7tzVd1fwkxsoryt...


In [38]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      50 non-null     object
 1   name          50 non-null     object
 2   release_date  50 non-null     object
 3   total_tracks  50 non-null     int64 
 4   url           50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [39]:
# Eliminating all duplicates on the basis of Album_id
album_df = album_df.drop_duplicates(subset=['album_id'])

In [40]:
# Repeating the same process for Artist dataframe

artist_df = pd.DataFrame.from_dict(artist_list)
artist_df = artist_df.drop_duplicates(subset=['artist_id'])

In [41]:
# Song Dataframe - process

song_df = pd.DataFrame.from_dict(song_list)

In [42]:
song_df.head()

,song_id,song_name,duration_ms,url,popularity,song_added,album_id,artist_id
0,0R6NfOiLzLj4O5VbYSJAjf,La Diabla,172264,https://open.spotify.com/track/0R6NfOiLzLj4O5V...,99,2024-01-26T16:09:07Z,2IoSjweCu07y9mv5H1Cj3v,3Me35AWHCGqW4sZ7bWWJt1
1,7gaA3wERFkFkgivjwbSvkG,"yes, and?",214994,https://open.spotify.com/track/7gaA3wERFkFkgiv...,96,2024-01-26T16:09:07Z,2B9amdrHDIKOoEiPUfZGtb,66CXWjxzNUsdJxJ2JdwvnR
2,3rUGC1vUpkDG9CZFHMur1t,greedy,131872,https://open.spotify.com/track/3rUGC1vUpkDG9CZ...,100,2024-01-26T16:09:07Z,3UOV8XvCwMKaATRNXrYCjN,45dkTj5sMRSjrmBSBeiHym
3,4xhsWYTOGcal8zt0J161CU,Lovin On Me,138411,https://open.spotify.com/track/4xhsWYTOGcal8zt...,99,2024-01-26T16:09:07Z,6VCO0fDBGbRW8mCEvV95af,2LIk90788K0zvyj2JJVwkJ
4,7CyPwkp0oE8Ro9Dd5CUDjW,"One Of The Girls (with JENNIE, Lily Rose Depp)",244684,https://open.spotify.com/track/7CyPwkp0oE8Ro9D...,98,2024-01-26T16:09:07Z,7tzVd1fwkxsorytCBjEJkU,1Xyo4u8uXC1ZmMpatF05PJ


In [43]:
artist_df.head()

,artist_id,artist_name,external_url
0,3Me35AWHCGqW4sZ7bWWJt1,Xavi,https://api.spotify.com/v1/artists/3Me35AWHCGq...
1,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,https://api.spotify.com/v1/artists/66CXWjxzNUs...
2,45dkTj5sMRSjrmBSBeiHym,Tate McRae,https://api.spotify.com/v1/artists/45dkTj5sMRS...
3,2LIk90788K0zvyj2JJVwkJ,Jack Harlow,https://api.spotify.com/v1/artists/2LIk90788K0...
4,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,https://api.spotify.com/v1/artists/1Xyo4u8uXC1...


### Data Type Conversion & DataFrame information

In [44]:
# Converting object datatype to datatime type
album_df['release_date'] = pd.to_datetime(album_df['release_date'])

In [45]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   album_id      47 non-null     object        
 1   name          47 non-null     object        
 2   release_date  47 non-null     datetime64[ns]
 3   total_tracks  47 non-null     int64         
 4   url           47 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.2+ KB


In [47]:
# Similarly converting object to datatime type for Song dataframe
song_df['song_added'] =  pd.to_datetime(song_df['song_added'])

song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   song_id      50 non-null     object             
 1   song_name    50 non-null     object             
 2   duration_ms  50 non-null     int64              
 3   url          50 non-null     object             
 4   popularity   50 non-null     int64              
 5   song_added   50 non-null     datetime64[ns, UTC]
 6   album_id     50 non-null     object             
 7   artist_id    50 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 3.2+ KB


### Next Process in the project: Deploying the Code into Cloud

- For Automation purposes 
- Storing the data to facilitate the execution of SQL queries upon it.